<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1DCqlw6PyG8XFjjdKRljSijwsKC1LRTAp?usp=sharing)

## Master Generative AI in 4 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 4 weeks of hands-on, project-based learning
- Join Innovation Community

Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)

# Gemini 3.0 Pro Structured Outputs with Google GenAI SDK

This notebook demonstrates how to use the new `google-genai` SDK to generate structured outputs (JSON/Pydantic) with Google's Gemini models. We'll explore various use cases combining **Google Search** and **Structured Outputs**.

**Prerequisites:**
- Google Cloud Project or AI Studio Account
- API Key from [Google AI Studio](https://aistudio.google.com/)

### 1. Installation
Install the experimental Google GenAI SDK and Pydantic.

In [1]:
%pip install google-genai pydantic

### 2. Setup & Authentication
Configure your API key. It is recommended to use Google Colab's `userdata` for security.

In [2]:
import os
from google import genai
from google.genai import types
from google.colab import userdata
from pydantic import BaseModel, Field
from typing import List, Optional

# Setup API Key
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
except ImportError:
    # Fallback if not running in Colab or userdata not set
    os.environ['GOOGLE_API_KEY'] = 'YOUR_API_KEY_HERE'

client = genai.Client(api_key=os.environ['GOOGLE_API_KEY'])

### 3. Example 1: Financial Data Extraction (Stock Analysis)
This example uses Google Search to find real-time stock information and formats it into a strict schema.

In [ ]:
# Define structured output schema
class StockData(BaseModel):
    ticker: str = Field(description="Stock ticker symbol (e.g., NVDA)")
    current_price: float = Field(description="Current stock price in USD")
    change_percent: float = Field(description="Today's percentage change")
    top_news: List[str] = Field(description="Top 3 news headlines affecting the stock")

# Make API call with Google Search + Structured Output
response = client.models.generate_content(
    model="gemini-3-pro-preview",  # Using latest 3.0 Pro Model
    contents="Get Nvidia's current stock price, today's percentage change, and top 3 news headlines affecting the stock",
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        response_mime_type="application/json",
        response_schema=StockData,
    ),
)

# Parse response into Pydantic model
result = StockData.model_validate_json(response.text)
print(f"Ticker: {result.ticker}")
print(f"Price: ${result.current_price} ({result.change_percent}%)")
print("News Headlines:")
for news in result.top_news:
    print(f"- {news}")

### 4. Example 2: Smart Product Comparison
Compare two products by gathering real-time specs, prices, and reviews using Google Search, returned as a structured comparison object.

In [ ]:
class ProductInfo(BaseModel):
    name: str
    estimated_price: str
    key_specs: List[str]
    pros: List[str]
    cons: List[str]

class ProductComparison(BaseModel):
    category: str
    products: List[ProductInfo]
    verdict: str = Field(description="Which product is better for which type of user")

prompt = "Compare the latest iPhone 15 Pro Max vs Samsung Galaxy S24 Ultra. Include current pricing and key differences."

response = client.models.generate_content(
    model="gemini-3-pro-preview",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        response_mime_type="application/json",
        response_schema=ProductComparison,
    ),
)

comparison = ProductComparison.model_validate_json(response.text)
print(f"=== {comparison.category} ===")
for p in comparison.products:
    print(f"\n{p.name} ({p.estimated_price})")
    print(f"Pros: {', '.join(p.pros[:2])}...")
    print(f"Specs: {', '.join(p.key_specs[:3])}...")

print(f"\nVerdict: {comparison.verdict}")

### 5. Example 3: Automated Travel Itinerary Planner
Generate a detailed travel plan. While search helps with accuracy, the model's knowledge can also be structured purely for planning.

In [ ]:
class DayPlan(BaseModel):
    day: int
    theme: str
    activities: List[str]
    restaurants: List[str]

class TravelItinerary(BaseModel):
    destination: str
    duration: str
    budget: str
    daily_plans: List[DayPlan]
    packing_tips: List[str]

prompt = "Plan a 3-day weekend trip to Kyoto, Japan for a couple who loves history and food. Budget is mid-range."

response = client.models.generate_content(
    model="gemini-3-pro-preview",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        response_mime_type="application/json",
        response_schema=TravelItinerary,
    ),
)

itinerary = TravelItinerary.model_validate_json(response.text)
print(f"Trip to {itinerary.destination} ({itinerary.duration})")
for day in itinerary.daily_plans:
    print(f"\nDay {day.day}: {day.theme}")
    print("  Activities:")
    for act in day.activities:
        print(f"  - {act}")
    print("  Dining:")
    for rest in day.restaurants:
        print(f"  - {rest}")

### 6. Example 4: Research & News Summarizer
Find the latest developments in a specific field and return them as a structured list of articles.

In [ ]:
class Article(BaseModel):
    title: str
    source: str
    date: str
    summary: str
    key_takeaway: str

class ResearchDigest(BaseModel):
    topic: str
    articles: List[Article]

prompt = "Find 3 recent breakthroughs in Solid State Battery technology from the last 6 months."

response = client.models.generate_content(
    model="gemini-3-pro-preview",
    contents=prompt,
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        response_mime_type="application/json",
        response_schema=ResearchDigest,
    ),
)

digest = ResearchDigest.model_validate_json(response.text)
print(f"Research Digest: {digest.topic}\n")
for article in digest.articles:
    print(f"Title: {article.title}")
    print(f"Source: {article.source} ({article.date})")
    print(f"Summary: {article.summary}")
    print(f"Takeaway: {article.key_takeaway}\n")